<a href="https://colab.research.google.com/github/Mic-Fundraiser/fundraiser-utility/blob/main/GeneraBuste_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gradio reportlab pandas requests

In [ ]:
import gradio as gr
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.units import cm
import pandas as pd
import io
import os
from zipfile import ZipFile
import requests

# Definizione delle dimensioni della pagina per un layout orizzontale
custom_page_width = 22 * cm  # Larghezza
custom_page_height = 11 * cm  # Altezza
custom_page_size = (custom_page_width, custom_page_height)

# Creazione della cartella temporanea per i PDF
output_folder = "pdf_outputs"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Funzione modificata per creare un PDF per ogni riga del dataframe
def create_individual_pdf(row, logo_path, sender_address, output_folder):
    output_pdf_path = f"{output_folder}/{row['Cognome']}_{row['Nome']}.pdf"
    margin = 1 * cm
    logo_size = 2 * cm  # Dimensione del logo
    text_size = 10
    sender_text_y = custom_page_height - margin - logo_size / 2  # Posizione del testo del mittente vicino al logo

    c = canvas.Canvas(output_pdf_path, pagesize=custom_page_size)
    c.setFont("Helvetica", 12)

    # Disegna il logo in alto a sinistra
    c.drawImage(logo_path, margin, custom_page_height - margin - logo_size, width=logo_size, height=logo_size)

    # Stampa l'indirizzo del mittente accanto al logo
    for i, line in enumerate(sender_address.split('\n')):
        c.drawString(margin + logo_size + 5, sender_text_y - (i * 15), line)

    # Posizione del nome e cognome del destinatario in basso a destra
    recipient_name_x = custom_page_width - margin - 6 * cm  # Spazio per il testo del destinatario
    recipient_name_y = margin + 2 * cm  # Posizione dal basso

    # Stampa l'indirizzo del destinatario
    c.drawString(recipient_name_x, recipient_name_y, f"{row['Cognome']} {row['Nome']}")
    c.drawString(recipient_name_x, recipient_name_y - 15, row['Indirizzo'])
    c.drawString(recipient_name_x, recipient_name_y - 30, f"{row['CAP']} {row['Città']} ({row['Provincia']})")

    c.showPage()
    c.save()

# Funzione per comprimere i file in un archivio ZIP e scaricare
def compress_files_to_zip(output_folder, zip_name):
    with ZipFile(zip_name, 'w') as zipf:
        for root, dirs, file_list in os.walk(output_folder):
            for file in file_list:
                zipf.write(os.path.join(root, file), arcname=file)
    return zip_name

# Funzione principale per Gradio
def process_and_download_csv(uploaded_csv, logo_url, nome_azienda, via_e_numero, cap, città, sigla_provincia):
    # Scaricamento e salvataggio del file del logo
    response = requests.get(logo_url)
    logo_path = f"{output_folder}/logo.png"
    with open(logo_path, 'wb') as f:
        f.write(response.content)

    # Caricamento del file CSV
    dataframe = pd.read_csv(uploaded_csv)

    # Indirizzo del mittente con andata a capo
    sender_address = f"{nome_azienda}\n{via_e_numero}\n{cap} {città} ({sigla_provincia})"

    # Iterazione su ogni riga del dataframe per creare un PDF per ogni destinatario
    for index, row in dataframe.iterrows():
        create_individual_pdf(row, logo_path, sender_address, output_folder)

    # Comprimere tutti i PDF in un file ZIP e restituirlo
    zip_name = "output_pdfs.zip"
    return compress_files_to_zip(output_folder, zip_name)

# Definizione dell'interfaccia Gradio con tutti gli input richiesti.
iface = gr.Interface(
    fn=process_and_download_csv,
    inputs=[
        gr.File(label="Carica il file CSV con i dati degli indirizzi"),
        gr.Text(label="Inserisci l'URL del logo (opzionale)"),
        gr.Text(label="Nome Azienda/Associazione"),
        gr.Text(label="Via e numero civico"),
        gr.Text(label="CAP"),
        gr.Text(label="Città"),
        gr.Text(label="Sigla Provincia")
    ],
    outputs=gr.File(label="Scarica lo zip con le buste da lettera"),  # Modificato per dimostrazione, sostituisci con gr.File per il download.
    title="Generatore di buste singole con logo, mittente e destinatario",
    description="Carica un file CSV con i dati degli indirizzi, inserisci l'URL del logo e dettaglia l'indirizzo del mittente per generare PDF delle buste da lettera e scaricare un ZIP contenente i PDF."
)

iface.launch(share=True, debug=True)
